In [1]:
# Cell 1: Setup and Imports
import sys
sys.path.append('..')  # To import from config folder

import requests
import pandas as pd
from pprint import pprint
from config.api_keys import GOOGLE_MAPS_API_KEY, YELP_API_KEY

print("✅ Imports successful!")
print(f"Google API Key: {GOOGLE_MAPS_API_KEY[:10]}... (hidden)")
print(f"Yelp API Key: {YELP_API_KEY[:10]}... (hidden)")

✅ Imports successful!
Google API Key: AIzaSyD-CT... (hidden)
Yelp API Key: GaMZRPGyIA... (hidden)


In [3]:
# Cell 2: Test Google Places API - Search for Museums in Boston
print("\n" + "="*60)
print("TESTING GOOGLE PLACES API")
print("="*60)

google_url = "https://places.googleapis.com/v1/places:searchText"

google_headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.rating,places.userRatingCount,places.priceLevel'
}

google_payload = {
    "textQuery": "museums in Boston",
    "maxResultCount": 5
}

try:
    response = requests.post(google_url, json=google_payload, headers=google_headers)
    print(f"\n✅ Status Code: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        print(f"✅ Found {len(data.get('places', []))} places")
        print("\nFirst result:")
        pprint(data.get('places', [])[0])
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"❌ Exception: {e}")


TESTING GOOGLE PLACES API

✅ Status Code: 200
✅ Found 5 places

First result:
{'displayName': {'languageCode': 'en', 'text': 'Museum of Illusions Boston'},
 'formattedAddress': 'Across from Faneuil Hall and Quincy Market, 200 State St '
                     '#2, Boston, MA 02109, USA',
 'id': 'ChIJLwYOQ0lx44kRCRT7HB0zn8Q',
 'rating': 4.7,
 'userRatingCount': 811}


In [4]:
# Cell 3: Convert Google Places Response to DataFrame
if response.status_code == 200:
    places_list = []
    for place in data.get('places', []):
        places_list.append({
            'name': place.get('displayName', {}).get('text', 'N/A'),
            'address': place.get('formattedAddress', 'N/A'),
            'rating': place.get('rating', None),
            'reviews': place.get('userRatingCount', 0),
            'price_level': place.get('priceLevel', 'N/A'),
            'source': 'google'
        })
    
    google_df = pd.DataFrame(places_list)
    print("\n📊 Google Places DataFrame:")
    print(google_df)
else:
    print("⚠️ Skipping DataFrame creation due to API error")


📊 Google Places DataFrame:
                                    name  \
0             Museum of Illusions Boston   
1        Isabella Stewart Gardner Museum   
2            Museum of Fine Arts, Boston   
3                      Museum of Science   
4  The Harvard Museum of Natural History   

                                             address  rating  reviews  \
0  Across from Faneuil Hall and Quincy Market, 20...     4.7      811   
1                25 Evans Way, Boston, MA 02115, USA     4.7    10343   
2          465 Huntington Ave, Boston, MA 02115, USA     4.8    18946   
3                1 Science Pk, Boston, MA 02114, USA     4.7    22456   
4             26 Oxford St, Cambridge, MA 02138, USA     4.8     5290   

            price_level  source  
0                   N/A  google  
1  PRICE_LEVEL_MODERATE  google  
2                   N/A  google  
3                   N/A  google  
4                   N/A  google  


In [5]:
# Cell 4: Test Yelp API - Search for Museums in Boston
print("\n" + "="*60)
print("TESTING YELP API")
print("="*60)

yelp_url = "https://api.yelp.com/v3/businesses/search"

yelp_headers = {
    'Authorization': f'Bearer {YELP_API_KEY}'
}

yelp_params = {
    'location': 'Boston, MA',
    'term': 'museums',
    'limit': 5
}

try:
    response = requests.get(yelp_url, headers=yelp_headers, params=yelp_params)
    print(f"\n✅ Status Code: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        print(f"✅ Found {len(data.get('businesses', []))} businesses")
        print("\nFirst result:")
        pprint(data.get('businesses', [])[0])
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"❌ Exception: {e}")


TESTING YELP API

✅ Status Code: 200
✅ Found 5 businesses

First result:
{'alias': 'boston-tea-party-ships-and-museum-boston',
 'attributes': {'business_temp_closed': None, 'waitlist_reservation': None},
 'business_hours': [{'hours_type': 'REGULAR',
                     'is_open_now': True,
                     'open': [{'day': 0,
                               'end': '1700',
                               'is_overnight': False,
                               'start': '1000'},
                              {'day': 1,
                               'end': '1700',
                               'is_overnight': False,
                               'start': '1000'},
                              {'day': 2,
                               'end': '1700',
                               'is_overnight': False,
                               'start': '1000'},
                              {'day': 3,
                               'end': '1700',
                               'is_overnight': Fal

In [6]:
# Cell 5: Convert Yelp Response to DataFrame
if response.status_code == 200:
    businesses_list = []
    for biz in data.get('businesses', []):
        businesses_list.append({
            'name': biz.get('name', 'N/A'),
            'address': ', '.join(biz.get('location', {}).get('display_address', [])),
            'rating': biz.get('rating', None),
            'reviews': biz.get('review_count', 0),
            'price': biz.get('price', 'N/A'),
            'source': 'yelp'
        })
    
    yelp_df = pd.DataFrame(businesses_list)
    print("\n📊 Yelp DataFrame:")
    print(yelp_df)
else:
    print("⚠️ Skipping DataFrame creation due to API error")


📊 Yelp DataFrame:
                              name                                   address  \
0  Boston Tea Party Ships & Museum         306 Congress St, Boston, MA 02210   
1              Museum of Fine Arts      465 Huntington Ave, Boston, MA 02115   
2  Isabella Stewart Gardner Museum            25 Evans Way, Boston, MA 02115   
3            Boston Public Library         700 Boylston St, Boston, MA 02116   
4                Museum of Bad Art  1250 Massachusetts Ave, Boston, MA 02125   

   rating  reviews price source  
0     4.5      652   N/A   yelp  
1     4.4     1017   N/A   yelp  
2     4.3      773   N/A   yelp  
3     4.6      642   N/A   yelp  
4     4.2       58   N/A   yelp  


In [7]:
# Cell 6: Compare Results Side by Side
print("\n" + "="*60)
print("COMPARISON")
print("="*60)

if 'google_df' in locals() and 'yelp_df' in locals():
    print("\n🔵 GOOGLE PLACES:")
    print(google_df[['name', 'rating', 'reviews']])
    
    print("\n🟡 YELP:")
    print(yelp_df[['name', 'rating', 'reviews']])
    
    print("\n📊 STATS:")
    print(f"Google avg rating: {google_df['rating'].mean():.2f}")
    print(f"Yelp avg rating: {yelp_df['rating'].mean():.2f}")
    print(f"Google total reviews: {google_df['reviews'].sum()}")
    print(f"Yelp total reviews: {yelp_df['reviews'].sum()}")


COMPARISON

🔵 GOOGLE PLACES:
                                    name  rating  reviews
0             Museum of Illusions Boston     4.7      811
1        Isabella Stewart Gardner Museum     4.7    10343
2            Museum of Fine Arts, Boston     4.8    18946
3                      Museum of Science     4.7    22456
4  The Harvard Museum of Natural History     4.8     5290

🟡 YELP:
                              name  rating  reviews
0  Boston Tea Party Ships & Museum     4.5      652
1              Museum of Fine Arts     4.4     1017
2  Isabella Stewart Gardner Museum     4.3      773
3            Boston Public Library     4.6      642
4                Museum of Bad Art     4.2       58

📊 STATS:
Google avg rating: 4.74
Yelp avg rating: 4.40
Google total reviews: 57846
Yelp total reviews: 3142


In [8]:
# Cell 7: Save Sample Data (Optional)
# Uncomment to save
google_df.to_csv('../data/google_sample.csv', index=False)
yelp_df.to_csv('../data/yelp_sample.csv', index=False)
print("✅ Sample data saved to data/ folder")

✅ Sample data saved to data/ folder


In [10]:
# price analysis

print("="*60)
print("PRICE DATA ANALYSIS")
print("="*60)

# Check Google Places price data
if 'google_df' in locals():
    print("\n🔵 GOOGLE PLACES - Price Levels:")
    print(google_df[['name', 'price_level']])
    print(f"\nPrice level breakdown:")
    print(google_df['price_level'].value_counts())

# Check Yelp price data
if 'yelp_df' in locals():
    print("\n🟡 YELP - Price Levels:")
    print(yelp_df[['name', 'price']])
    print(f"\nPrice breakdown:")
    print(yelp_df['price'].value_counts())

PRICE DATA ANALYSIS

🔵 GOOGLE PLACES - Price Levels:
                                    name           price_level
0             Museum of Illusions Boston                   N/A
1        Isabella Stewart Gardner Museum  PRICE_LEVEL_MODERATE
2            Museum of Fine Arts, Boston                   N/A
3                      Museum of Science                   N/A
4  The Harvard Museum of Natural History                   N/A

Price level breakdown:
price_level
N/A                     4
PRICE_LEVEL_MODERATE    1
Name: count, dtype: int64

🟡 YELP - Price Levels:
                              name price
0  Boston Tea Party Ships & Museum   N/A
1              Museum of Fine Arts   N/A
2  Isabella Stewart Gardner Museum   N/A
3            Boston Public Library   N/A
4                Museum of Bad Art   N/A

Price breakdown:
price
N/A    5
Name: count, dtype: int64
